# 4.0 Evaluación y Análisis de Resultados

En este notebook, realizaremos un análisis más profundo de los resultados de los modelos entrenados y visualizaremos el rendimiento del modelo seleccionado en el conjunto de prueba.

In [1]:
from pathlib import Path
import sys

current_working_directory = Path.cwd()
project_root = current_working_directory.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
# Bibliotecas necesarias
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils.helpers import print_rmse, train_pred_time, test_parameters
from src.data.make_dataset import load_and_resample_taxi_data
from src.features.build_features import make_features
from src.models.train_model import train_best_model 
from src.models.predict_model import load_model, make_predictions
from src.visualization.visualize import plot_model_comparison, plot_test_predictions, plot_time_series


### Carga de Datos y Modelos

Cargamos el DataFrame original (remuestreado), el modelo final entrenado y las predicciones del conjunto de prueba.

In [5]:
processed_data_path = '../data/processed/resampled_taxi_data.parquet'
features_data_path = '../data/processed/features_taxi_data.parquet'
model_path = '../models/best_taxi_demand_model.pkl'
predictions_test_path = '../data/processed/test_predictions.parquet'

# Cargar datos originales remuestreados
if os.path.exists(processed_data_path):
    df_resampled = pd.read_parquet(processed_data_path)
    print("DataFrame remuestreado cargado.")
else:
    print(f"ADVERTENCIA: No se encontró {processed_data_path}. Ejecute el notebook 1.0.")
    df_resampled = None

# Cargar el DataFrame con características
if os.path.exists(features_data_path):
    df_features = pd.read_parquet(features_data_path)
    print("DataFrame con características cargado.")
else:
    print(f"ADVERTENCIA: No se encontró {features_data_path}. Ejecute el notebook 2.0.")
    df_features = None

# Cargar el modelo entrenado
if os.path.exists(model_path):
    final_model = load_model(model_path)
else:
    print(f"ADVERTENCIA: No se encontró el modelo en {model_path}. Ejecute el notebook 3.0.")
    final_model = None

# Cargar las predicciones del conjunto de prueba
if os.path.exists(predictions_test_path):
    predictions_test_df = pd.read_parquet(predictions_test_path)
    print("Predicciones del conjunto de prueba cargadas.")
    target_test = predictions_test_df['real']
    predictions_test_values = predictions_test_df['predicted']
else:
    print(f"ADVERTENCIA: No se encontraron las predicciones de prueba en {predictions_test_path}. Ejecute el notebook 3.0.")
    target_test = None
    predictions_test_values = None

DataFrame remuestreado cargado.
DataFrame con características cargado.
Cargando modelo desde: ../models/best_taxi_demand_model.pkl
Modelo cargado exitosamente.
Predicciones del conjunto de prueba cargadas.


### Comparación de Métricas de Modelos

In [ ]:
if df_features is not None:
    df_model_ready = df_features.dropna().copy()

    features_all = df_model_ready.drop(['num_orders'], axis=1)
    target_all = df_model_ready['num_orders']

    features_train, features_test, target_train, target_test_comparison = train_test_split(
        features_all, target_all, shuffle=False, test_size=0.1
    )
    
    data_models = []
    STATE = 31415

    print("Iniciando re-evaluación de todos los modelos para comparación...")

    # --- Linear Regression ---
    print("\n--- Modelo de Regresión Lineal ---")
    from sklearn.linear_model import LinearRegression
    model_lr = LinearRegression()
    times_lr, predictions_lr = train_pred_time(model_lr, features_train, target_train, features_test)
    rmse_train_lr = print_rmse(target_train, predictions_lr['train'])
    rmse_test_lr = print_rmse(target_test_comparison, predictions_lr['test'])
    data_models.append(['linear_regression', times_lr['train'], times_lr['predict'], rmse_train_lr, rmse_test_lr])

    # --- Decision Tree ---
    print("\n--- Modelo de Árbol de Decisiones ---")
    from sklearn.tree import DecisionTreeRegressor
    model_dt = DecisionTreeRegressor(random_state=STATE)
    param_grid_dt = {'max_depth': list(range(2, 51, 2))}
    best_params_dt = test_parameters(model_dt, param_grid_dt, features_train, target_train)
    model_dt_best = DecisionTreeRegressor(**best_params_dt, random_state=STATE)
    times_dt, predictions_dt = train_pred_time(model_dt_best, features_train, target_train, features_test)
    rmse_train_dt = print_rmse(target_train, predictions_dt['train'])
    rmse_test_dt = print_rmse(target_test_comparison, predictions_dt['test'])
    data_models.append(['desicion_tree', times_dt['train'], times_dt['predict'], rmse_train_dt, rmse_test_dt])

    # --- Random Forest ---
    print("\n--- Modelo de Bosque Aleatorio ---")
    from sklearn.ensemble import RandomForestRegressor
    model_rf = RandomForestRegressor(random_state=STATE)
    param_grid_rf = {'n_estimators': [30, 40], 'max_depth': [10, 20]} # Reducir para faster execution in notebook
    best_params_rf = test_parameters(model_rf, param_grid_rf, features_train, target_train)
    model_rf_best = RandomForestRegressor(**best_params_rf, random_state=STATE)
    times_rf, predictions_rf = train_pred_time(model_rf_best, features_train, target_train, features_test)
    rmse_train_rf = print_rmse(target_train, predictions_rf['train'])
    rmse_test_rf = print_rmse(target_test_comparison, predictions_rf['test'])
    data_models.append(['random_forest', times_rf['train'], times_rf['predict'], rmse_train_rf, rmse_test_rf])

    # --- LightGBM ---
    print("\n--- Modelo LightGBM ---")
    import lightgbm as lgb
    model_lgbm = lgb.LGBMRegressor(boosting_type='gbdt', random_state=STATE, force_row_wise=True, verbose=-1)
    param_grid_lgbm = {
        'max_depth': [10, 20], # Reducir para faster execution
        'num_leaves': [10, 20],
        'n_estimators': [100, 500],
        'learning_rate': [0.1, 0.2]
    }
    best_params_lgbm = test_parameters(model_lgbm, param_grid_lgbm, features_train, target_train)
    model_lgbm_best = lgb.LGBMRegressor(**best_params_lgbm, boosting_type='gbdt', random_state=STATE, force_row_wise=True, verbose=-1)
    times_lgbm, predictions_lgbm = train_pred_time(model_lgbm_best, features_train, target_train, features_test)
    rmse_train_lgbm = print_rmse(target_train, predictions_lgbm['train'])
    rmse_test_lgbm = print_rmse(target_test_comparison, predictions_lgbm['test'])
    data_models.append(['lgbm', times_lgbm['train'], times_lgbm['predict'], rmse_train_lgbm, rmse_test_lgbm])

    # --- CatBoost ---
    print("\n--- Modelo CatBoost ---")
    from catboost import CatBoostRegressor
    model_cb = CatBoostRegressor(random_state=STATE, verbose=False, loss_function='RMSE')
    param_grid_cb = {
        'iterations': [100, 500], # Reducir para faster execution
        'learning_rate': [0.01, 0.1],
        'depth': [5, 10],
    }
    best_params_cb = test_parameters(model_cb, param_grid_cb, features_train, target_train)
    model_cb_best = CatBoostRegressor(**best_params_cb, random_state=STATE, verbose=False, loss_function='RMSE')
    times_cb, predictions_cb = train_pred_time(model_cb_best, features_train, target_train, features_test)
    rmse_train_cb = print_rmse(target_train, predictions_cb['train'])
    rmse_test_cb = print_rmse(target_test_comparison, predictions_cb['test'])
    data_models.append(['catboost', times_cb['train'], times_cb['predict'], rmse_train_cb, rmse_test_cb])

    # --- XGBoost ---
    print("\n--- Modelo XGBoost ---")
    import xgboost as xgb
    model_xgb = xgb.XGBRegressor(random_state=STATE, eval_metric='rmse')
    param_grid_xgb = {
        'n_estimators': [100, 500], # Reducir para faster execution
        'max_depth': [5, 10],
        'learning_rate': [0.01, 0.1],
    }
    best_params_xgb = test_parameters(model_xgb, param_grid_xgb, features_train, target_train)
    model_xgb_best = xgb.XGBRegressor(**best_params_xgb, random_state=STATE, eval_metric='rmse')
    times_xgb, predictions_xgb = train_pred_time(model_xgb_best, features_train, target_train, features_test)
    rmse_train_xgb = print_rmse(target_train, predictions_xgb['train'])
    rmse_test_xgb = print_rmse(target_test_comparison, predictions_xgb['test'])
    data_models.append(['xgboost', times_xgb['train'], times_xgb['predict'], rmse_train_xgb, rmse_test_xgb])

    models_df = pd.DataFrame(data_models, columns=['model', 'training_time', 'prediction_time', 'rmse_train', 'rmse_test'])
    print("\nTabla de Comparación de Modelos:")
    print(models_df)

    # Generar gráfico de comparación
    plot_model_comparison(models_df, save_path='reports/figures/model_comparison_metrics_times.png')

    print("\nObservaciones destacables:")
    print("- El modelo con mejor RMSE más bajo corresponde a Catboost, aunque también corresponde al que posee un mayor tiempo de entrenamiento y un tiempo de pruebas medio.")
    print("- El segundo mejor es LightGBM, cuya diferencia en las métricas es de aproximadamente 0.2, con la diferencia de que el tiempo de entrenamiento es notablemente más bajo y el de pruebas solo un poco más alto.")
    print("- Todos los modelos, a excepción del árbol de decisiones, superaron el mínimo requerido para ser utilizados, incluyendo el de regresión lineal.")

else:
    print("No se pudo realizar la comparación de modelos porque los datos de características no se cargaron.")
    models_df = None